In [1]:
# 03_model_training.ipynb

# Step 1: Setup & Imports
import os
import sys
import matplotlib.pyplot as plt

from pathlib import Path
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Project path
sys.path.append("..")

from src.model_config import ModelConfig
from src.model_architecture import LeukemiaDetectionModel
from src.data_preprocessing import DataLoader
from src.data_augmentation import get_augmentation_pipeline
from src.utils import (
    setup_logging, print_system_info,
    plot_training_history, save_model_history,
    create_experiment_folder
)

# Step 2: Configuration
config = ModelConfig()
model_params = config.get_model_params()
training_params = config.get_training_params()
data_params = config.get_data_params()

# Logging
setup_logging(log_level="INFO")
print_system_info()

# Step 3: Load Data
loader = DataLoader(
    dataset_path=data_params["dataset_path"],
    processed_path=data_params["processed_path"],
    image_size=tuple(data_params["image_size"]),
    seed=data_params["random_seed"]
)

train_ds, val_ds, test_ds, class_names = loader.get_tf_datasets(
    batch_size=data_params["batch_size"]
)

# Optional: Apply augmentation
augmenter = get_augmentation_pipeline(config.augmentation_config)
train_ds = train_ds.map(lambda x, y: (augmenter(x, training=True), y))

# Step 4: Create Output Folder
experiment_path = create_experiment_folder()
checkpoint_path = Path(config.paths_config["checkpoint_path"]) / "best_model.h5"
history_path = Path(experiment_path) / "history.json"
plot_path = Path(experiment_path) / "training_plot.png"

# Step 5: Build Model
model = LeukemiaDetectionModel(config).build_model()
model.summary()

# Step 6: Compile Model
model.compile(
    optimizer=training_params["optimizer"],
    loss=training_params["loss_function"],
    metrics=training_params["metrics"]
)

# Step 7: Define Callbacks
callbacks = [
    EarlyStopping(
        patience=training_params["early_stopping_patience"],
        restore_best_weights=True,
        monitor='val_loss'
    ),
    ReduceLROnPlateau(
        factor=training_params["reduce_lr_factor"],
        patience=training_params["reduce_lr_patience"],
        monitor='val_loss',
        verbose=1
    ),
    ModelCheckpoint(
        filepath=str(checkpoint_path),
        save_best_only=True,
        monitor='val_loss',
        verbose=1
    )
]

# Step 8: Train the Model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=training_params["epochs"],
    callbacks=callbacks
)

# Step 9: Save Training History and Plot
save_model_history(history, str(history_path))

plot_training_history(
    {
        'epoch': list(range(1, len(history.history['loss']) + 1)),
        **history.history
    },
    save_path=str(plot_path)
)

print(f"\n✅ Training complete. History saved at: {history_path}")
print(f"📊 Training plots saved at: {plot_path}")
print(f"📦 Best model saved at: {checkpoint_path}")

FileNotFoundError: Configuration file not found: c:\Users\HP\OneDrive\Desktop\leukemia_detection\notebooks\..\src\config.yaml